In [14]:
import cv2
import numpy as np
import glob
from csc.charuco_stereo_calibrator import CharucoStereoCalibrator

In [15]:
# Example usage
images_left = glob.glob("samples/checker/*.jpg")
images_right = glob.glob("samples/checker_r/*.jpg")

chessboard_size = (11, 8)
frame_size_h = 2592 // 2
frame_size_w = 4608 // 2

# if below is None, then algorithm figure this out.
f_in_mm = 4.74
pixel_size_mm = 1.4e-3 * 2  # binning

stereo_calibrator = CharucoStereoCalibrator(
    chessboard_size=chessboard_size,
    frame_size_h=frame_size_h,
    frame_size_w=frame_size_w,
    f_in_mm=f_in_mm,
    pixel_size_mm=pixel_size_mm,
    debug=False,
)

[INFO] 2024-11-30 19:15:07 - Calibration started for stereo images.


In [16]:
stereo_calibrator.perform_calibration(images_left, images_right)

[INFO] 2024-11-30 19:15:07 - Starting stereo image processing...

✅KNOWN CAMERA INTRINSICS:

[[1692.8571 0.0000 1152.0000]
 [0.0000 1692.8571 648.0000]
 [0.0000 0.0000 1.0000]]

[INFO] 2024-11-30 19:15:10 - Calibrating the left camera...
[SUCCESS] 2024-11-30 19:15:10 - 🎥 Left Camera Calibration RMS Error: 1.2554

LEFT CAMERA MATRIX:

[[1692.8571 0.0000 1152.0000]
 [0.0000 1692.8571 648.0000]
 [0.0000 0.0000 1.0000]]

[INFO] 2024-11-30 19:15:10 - Calibrating the right camera...
[SUCCESS] 2024-11-30 19:15:10 - 🎥 Right Camera Calibration RMS Error: 1.2759

RIGHT CAMERA MATRIX:

[[1692.8571 0.0000 1152.0000]
 [0.0000 1692.8571 648.0000]
 [0.0000 0.0000 1.0000]]

[INFO] 2024-11-30 19:15:10 - Performing stereo calibration...
[SUCCESS] 2024-11-30 19:15:11 - 🎥 Stereo Camera Calibration RMS Error: 1.4890
[INFO] 2024-11-30 19:15:11 - Saving calibration matrices and parameters...
Saving parameters!
All parameters saved successfully!
[SUCCESS] 2024-11-30 19:15:12 - Stereo calibration completed suc

In [17]:
print("0,0: ", stereo_calibrator.objpointsL[0][0])
print("0,1: ", stereo_calibrator.objpointsL[0][1])
print("0,2: ", stereo_calibrator.objpointsL[0][2])
print("1,0: ", stereo_calibrator.objpointsL[1][0])
print("==" * 10)
print("0,0: ", stereo_calibrator.objpointsR[0][0])
print("0,1: ", stereo_calibrator.objpointsR[0][1])
print("0,2: ", stereo_calibrator.objpointsR[0][2])
print("1,0: ", stereo_calibrator.objpointsR[1][0])

0,0:  [[20. 20.  0.]]
0,1:  [[40. 20.  0.]]
0,2:  [[60. 20.  0.]]
1,0:  [[20. 20.  0.]]
0,0:  [[20. 20.  0.]]
0,1:  [[40. 20.  0.]]
0,2:  [[60. 20.  0.]]
1,0:  [[20. 20.  0.]]


distance based on `square_mm` of the checker board

In [18]:
ret, camera_matrix, dist, rvecs, tvecs = stereo_calibrator.calibrate_camera(
    stereo_calibrator.objpointsL, stereo_calibrator.imgpointsL
)

- **ret**:
  - **Description**: The `ret` value, also known as the reprojection error, measures how well the calibration algorithm approximates the actual camera parameters. It is the average distance between observed image points and projected object points using estimated parameters.
  - **Mathematical Representation**:

    $$  
    \text{ret} = \frac{1}{N} \sum_{i=1}^{N} \| \mathbf{p}_i - \mathbf{\hat{p}}_i \|
    $$

    where:
    - $N$ is the total number of points,
    - $ \mathbf{p}_i $ is the observed image point,
    - $ \mathbf{\hat{p}}_i $ is the projected object point.

In [19]:
ret

1.2553738901379743

- **camera_matrix**:
  - **Description**: The `camera_matrix` $ K $ represents the camera's intrinsic parameters, including focal lengths and the optical center. It is a 3x3 matrix.
  - **Mathematical Representation**:

    $$ 
    K = \begin{bmatrix}
    f_x & 0 & c_x \\
    0 & f_y & c_y \\
    0 & 0 & 1
    \end{bmatrix}
    $$

    where:
    - $ f_x, f_y $ are the focal lengths in pixels,
    - $ c_x, c_y $ are the optical center coordinates in the image.

In [20]:
camera_matrix

array([[1.69285714e+03, 0.00000000e+00, 1.15200000e+03],
       [0.00000000e+00, 1.69285714e+03, 6.48000000e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

- **dist**:
  - **Description**: The `dist` array contains distortion coefficients for lens distortion, including radial and tangential distortions.
  - **Mathematical Representation**:

    $$ 
    \text{dist} = \begin{bmatrix}
    k_1 & k_2 & p_1 & p_2 & k_3
    \end{bmatrix}
    $$

    where:
    - $ k_1, k_2, k_3 $ are radial distortion coefficients,
    - $ p_1, p_2 $ are tangential distortion coefficients.

In [21]:
dist

array([[ 0.021592  ,  0.04840796,  0.00154551, -0.00355661, -0.18525454]])

- **rvecs**:
  - **Description**: `rvecs` (rotation vectors) represent the camera's rotation relative to each view of the calibration object. Each vector can be converted into a rotation matrix using Rodrigues' formula.
  - **Mathematical Representation**: Each $ \mathbf{rvec}_i $ is a 3-element vector for rotation in 3D space.

In [22]:
print(f"{len(rvecs)} vectors from 22 calibration images")
print("first rvec: \n", rvecs[0])

22 vectors from 22 calibration images
first rvec: 
 [[-0.61295532]
 [ 0.07442224]
 [-0.00826071]]


- **tvecs**:
  - **Description**: `tvecs` (translation vectors) represent the camera's translation relative to each view of the calibration object. They translate the origin of the world coordinate system to the camera coordinate system.
  - **Mathematical Representation**: Each $ \mathbf{tvec}_i $ is a 3-element vector for translation in 3D space.

In [23]:
print("first tvec: \n", tvecs[0])

first tvec: 
 [[-149.96290099]
 [ -86.8913117 ]
 [ 431.1844847 ]]


**Given**:
   - Intrinsic camera matrices $ K_1 $ and $ K_2 $ for two cameras.
   - Rotation matrix $ R $ and translation vector $ \mathbf{t} $ between the two

In [24]:
ret_R, camera_matrix_R, dist_R, rvecs_R, tvecs_R = stereo_calibrator.calibrate_camera(
    stereo_calibrator.objpointsR, stereo_calibrator.imgpointsR
)

### Fundamental Matrix (F)
The Fundamental Matrix $F$ is a $3 \times 3$ matrix that encapsulates the intrinsic and extrinsic parameters of the stereo camera system. It relates corresponding points between two images:
$$ \mathbf{x'}^T \cdot \mathbf{F} \cdot \mathbf{x} = 0 $$
where $\mathbf{x}$ and $\mathbf{x'}$ are homogeneous coordinates of corresponding points in left and right images, respectively.

Let's now compute the Fundamental Matrix $F$ and the Essential Matrix $E$ using the calibrated parameters.

In [35]:
# Typically object points are the same for both cameras in stereo calibration
imgpointsL = [np.array(pts, dtype=np.float32) for pts in stereo_calibrator.imgpointsL]
imgpointsR = [np.array(pts, dtype=np.float32) for pts in stereo_calibrator.imgpointsR]

# Example using a pair of points for each image (for demo purpose, ensure all points are used appropriately)
assert len(imgpointsL) == len(
    imgpointsR
), "The number of point sets must be equal for the left and right images."

# Now calculate the Fundamental Matrix using one set of corresponding points
F, mask = cv2.findFundamentalMat(imgpointsL[0], imgpointsR[0], cv2.FM_LMEDS)

# Print the matrices
print("Fundamental Matrix (F):\n", F)
print("mask shape: ", mask.shape)

Fundamental Matrix (F):
 [[ 5.81599683e-06 -1.14809833e-03  1.32732000e+00]
 [ 1.10956131e-03 -1.36141344e-04 -3.37529089e+00]
 [-1.23598760e+00  3.35822789e+00  1.00000000e+00]]
mask shape:  (70, 1)


The general form of the Fundamental Matrix is:

$$ F = \begin{bmatrix}
f_{11} & f_{12} & f_{13} \\
f_{21} & f_{22} & f_{23} \\
f_{31} & f_{32} & f_{33}
\end{bmatrix} $$

Each element of this matrix corresponds to a component of the transformation between corresponding points in the two images.

1. **General Interpretation**: 
   - $F$  is generated using matched image points and is fundamental in describing the epipolar lines between two images.
   - If a point $ \mathbf{x} $ in the first image corresponds to a point $ \mathbf{x'} $ in the second image, then $ \mathbf{x'}^T F \mathbf{x} = 0 $ must hold.

2. **Matrix Elements**:
   - The elements $ f_{ij} $ of the matrix do not have direct physical meaning like camera parameters. Instead, they collectively encode the epipolar geometry between two views.
   - The matrix is rank 2, meaning its determinant is zero, which is a fundamental property used in its derivation.

3. **Role in Epipolar Geometry**:
   - The rows and columns of $ F $ can be used to compute epipolar lines.
   - For a point $\mathbf{x}$ in the left image, the line $ \mathbf{l'} = F \mathbf{x} $ is the corresponding epipolar line in the right image.
   - Similarly, for a point $\mathbf{x'}$ in the right image, the line $ \mathbf{l} = F^T \mathbf{x'} $ is the corresponding epipolar line in the left image.

- **Mask Shape**: The mask's shape is $ (70, 1) $, indicating that there are 70 pairs of matched points. Each entry in the mask specifies whether the corresponding point pair was considered an inlier (1) or outlier (0) during the robust estimation process (e.g., using RANSAC).

### Essential Matrix (E)
The Essential Matrix $E$ relates corresponding points in a normalized (undistorted) image plane and reflects the extrinsic geometry between the cameras:
$$ \mathbf{x'}^T \cdot \mathbf{E} \cdot \mathbf{x} = 0 $$
The Essential Matrix is derived from the Fundamental Matrix when camera parameters are known, using:
$$ \mathbf{E} = \mathbf{K'}^T \cdot \mathbf{F} \cdot \mathbf{K} $$
where $\mathbf{K}$ and $\mathbf{K'}$ are the intrinsic parameters (camera matrices) of the left and right cameras.

In [31]:
# Ensure camera matrices are correctly defined
camera_matrix_L = np.array(camera_matrix)
camera_matrix_R = np.array(camera_matrix_R)

# Compute the Essential Matrix
E = camera_matrix_R.T @ F @ camera_matrix_L
print("Essential Matrix (E):\n", E)

Essential Matrix (E):
 [[   16.66728194 -3290.18034966   998.87427707]
 [ 3179.74230338  -390.14914154 -3699.39370094]
 [ -863.85119723  3296.66836033    16.94250883]]


$$ E = \begin{bmatrix}
e_{11} & e_{12} & e_{13} \\
e_{21} & e_{22} & e_{23} \\
e_{31} & e_{32} & e_{33}
\end{bmatrix} $$

Unlike the Fundamental Matrix, the Essential Matrix has direct geometric significance because it is computed in the normalized camera coordinate space.

- $ E $ is also a rank 2 matrix, indicating that it encapsulates a planar constraint in three-dimensional space.
- The matrix should satisfy the condition that the two non-zero singular values are equal when decomposed using SVD (Singular Value Decomposition).

In [ ]:
# Following calibrated parameters were obtained
R = stereo_calibrator.rot  # Rotation matrix from stereo calibration
T = stereo_calibrator.trans  # Translation vector from stereo calibration
camera_matrix_L = camera_matrix  # Camera matrix for the left camera
dist_coeffs_L = dist  # Distortion coefficients for the left camera
camera_matrix_R = camera_matrix_R  # Camera matrix for the right camera
dist_coeffs_R = dist_R  # Distortion coefficients for the right camera
image_size = (frame_size_w, frame_size_h)  # Size of the images

# Stereo rectification
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    camera_matrix_L,
    dist_coeffs_L,
    camera_matrix_R,
    dist_coeffs_R,
    image_size,
    R,
    T,
    alpha=0,
)

1. **Rectification Transforms $R1, R2$**:
   - $R1$: The rectification transformation for the first (left) camera.
   - $R2$: The rectification transformation for the second (right) camera.
   - **Purpose**: These matrices transform the original camera frames such that the epipolar lines become parallel and aligned along the horizontal axis. They adjust the orientation of each camera's image plane for rectification.

2. **Projection Matrices $P1, P2$**:
   - $P1$: The new projection matrix for the first camera after rectification.
   - $P2$: The new projection matrix for the second camera after rectification.
   - **Purpose**: These matrices define the perspective projection of 3D points into the rectified image frames. They account for the transformation applied through the rectification process and include adjustments for focal lengths and principal points, effectively detailing how the 3D world projects into each rectified image.

3. **Disparity-to-Depth Mapping Matrix $Q$**:
   - $Q$: A $4 \times 4$ reprojection matrix.
   - **Purpose**: This matrix is used to convert disparity values (differences in x-coordinates of corresponding points in the two images) back to 3D coordinates in real-world space. It essentially maps the 2D disparity from the rectified stereo images into 3D space, facilitating depth estimation and point cloud generation.

4. **Regions of Interest $roi1, roi2$**:
   - $roi1$: The valid region of interest in the rectified image for the first camera.
   - $roi2$: The valid region of interest in the rectified image for the second camera.
   - **Purpose**: These are the bounding rectangles that define the valid part of each rectified image where meaningful data exists post-rectification. It is useful for cropping out the borders of the rectified images that may not have valid information due to the lens distortion correction and image transformation.

### Summary of Use

- **Rectification Transforms ($R1, R2$)**: Aligns the stereo images for parallel epipolar lines.
- **Projection Matrices ($P1, P2$)**: Adjusts the perspective projection to account for rectification.
- **Disparity-to-Depth Mapping Matrix ($Q$)**: Converts disparity into 3D depth information.
- **Regions of Interest ($roi1, roi2$)**: Defines usable regions in the rectified images.

These components facilitate the preparation of stereo images for accurate stereo matching and depth estimation, ensuring that relevant geometrical constraints are satisfied.

---

### Dive deeper in $Q$ Matrix

The $Q$ matrix is derived from the relationship between the two rectified images and serves to transform the disparity value of a pixel into its real-world 3D coordinates. The concept hinges on understanding the geometry of stereo vision.

### Geometry of Stereo Vision

In stereo vision, each point in 3D space is projected onto two image planes. The disparity value for a point is the horizontal distance (in pixels) between its coordinates on the left and right images. Mathematically, if $(x_L, y_L)$ is the point in the left image and $(x_R, y_R)$ its corresponding point in the right image, then disparity $d$ is:

$$ d = x_L - x_R $$

### Reprojection with $Q$

The $Q$ matrix transforms a point in the disparity image into a 3D point as follows:

For a given pixel in the stereo images, the coordinates in the 3D world are calculated using:

$$
\begin{bmatrix}
X \\
Y \\
Z \\
W
\end{bmatrix}
=
Q
\begin{bmatrix}
x' \\
y' \\
d \\
1
\end{bmatrix}
$$

Where:
- $(x', y')$ are the pixel coordinates in the rectified image.
- $d$ is the disparity.

After transformation, the real-world coordinates $(X, Y, Z)$ are obtained by normalizing the resulting homogeneous coordinates, using $W$:

$$ X = \frac{X'}{W}, \quad Y = \frac{Y'}{W}, \quad Z = \frac{Z'}{W} $$

### Components of $Q$

The matrix $Q$ typically has the following structure:

$$
Q = \begin{bmatrix}
1 & 0 & 0 & -c_x \\
0 & 1 & 0 & -c_y \\
0 & 0 & 0 & f \\
0 & 0 & -\frac{1}{T} & \left( \frac{c_x - c_x'}{T} \right)
\end{bmatrix}
$$

Where:

- $c_x, c_y$ are the coordinates of the principal point in the left camera.
- $c_x'$ is the principal point in the right camera.
- $f$ is the focal length (assuming the same for both cameras after rectification).
- $T$ is the baseline distance between the two cameras.

### Key Relationships

- **Depth (Z-coordinate)**: Inversely proportional to disparity $d$. As the disparity decreases (objects are further and appear closer together in the images), the depth $Z$ increases.

- **Horizontal and Vertical Coordinates (X and Y)**: Scaled to the principal point offsets, ensuring that measurements are translated into the correct perspective based on the camera geometry.

This mathematical framework allows us to take a pixel location and its disparity from the stereo image pair and map them into real-world coordinates, forming the basis for 3D reconstruction and depth mapping in stereo vision systems. The precision and configuration of $Q$ are critical for accurate 3D modeling and understanding of the spatial environment captured by stereo cameras.

In [44]:
# Compute the undistortion and rectification transformation map
map1_L, map2_L = cv2.initUndistortRectifyMap(
    camera_matrix_L, dist_coeffs_L, R1, P1, image_size, cv2.CV_16SC2
)
map1_R, map2_R = cv2.initUndistortRectifyMap(
    camera_matrix_R, dist_coeffs_R, R2, P2, image_size, cv2.CV_16SC2
)
print("map1_L: ", map1_L.shape)
print("map2_L: ", map2_L.shape)

map1_L:  (1296, 2304, 2)
map2_L:  (1296, 2304)


- **Transformation Maps:**
   - $ \text{map1\_L}, \text{map2\_L} $ and $ \text{map1\_R}, \text{map2\_R} $ are the outputs for the left and right images, respectively. They are required by the `cv2.remap` function to actually perform the pixels' transformation.
   - Each map consists of two components. 
     - `map1` contains the (x, y) coordinates in the source image from which pixels should be taken
     - `map2` contains coefficients to compute the relative location with subpixel accuracy

#### Purpose

The primary purpose of these mappings is to correct distortions and apply the rectification transformations computed earlier. Applying these maps to the original images results in undistorted and rectified images where the epipolar geometry is simplified. This is crucial for accurate stereo correspondence searching, ultimately leading to more precise disparity maps and 3D reconstructions.

In [40]:
# Load the stereo images (replace with actual paths)
image_left = cv2.imread("left_image.jpg")
image_right = cv2.imread("right_image.jpg")

# Apply the rectification
rectified_left = cv2.remap(image_left, map1_L, map2_L, cv2.INTER_LINEAR)
rectified_right = cv2.remap(image_right, map1_R, map2_R, cv2.INTER_LINEAR)